<a href="https://colab.research.google.com/github/Camilo4004/FT-M0/blob/master/BBVA_User_Intent_Framework_Google_SERP_Scraper_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loop búsquedas relacionadas SERP
Ideas de contenido semántico y su enlazado interno sacado directamente de google

1. Pulsa al play en el primer módulo
2. En el segundo Módulo (abajo), cambia la keyword principal (ej. keyword = "enlazado interno SEO") y dale al play

↓↓ Los resultados salen debajo del script. Al final saldrán 2 tablas resumen ↓↓


**↓↓ 1. Pulsa al play ↓↓**

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver



Executing: /tmp/apt-key-gpghome.yhVN3kjaNp/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.QGwRSgn5Rl/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.lQQDVrgI0b/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian bust

**↓↓ 2. Cambia la keyword principal (ej. keyword = "enlazado interno SEO") y dale al play ↓↓**

In [ ]:
#HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH
keyword= 'tarjeta credito'  #  cambia la keyword por la que quieras, por ejemplo keyword = 'comprar backlink'
#HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH
# Parámetros opcionales
idioma = 'es' # español => por ejemplo para inglés usar idioma = 'en'
pais = 'ar' # España => por ejemplo para Inglaterra usar pais = 'uk'
nivelesScrapeo = 2 # Si quieres más resultados usa 2. A partir de 3 necesitarás proxies...
loopPreguntasEnlugarDeBusquedas = True # True hace loop pusando preguntas PAA en lugar de hacer el loop con búsquedas relacionadas
#HHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHHH

#!pip install pyopenSSl --upgrade
!pip install selenium-wire
from seleniumwire import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
#!apt update
#!apt install chromium-chromedriver

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("user-agent=Mozilla/5.0")
options.add_argument("--window-size=1366,768")#360,851

# iniciamos selenium
wd = webdriver.Chrome(options=options)


urlInicial = f"https://www.google.com/search?hl={idioma}&gl={pais}&q={keyword}&oq={keyword}"

scrapeado = []
contadorPreguntas = []
contadorBusquedas = []
contadorSuggest = []

googleNosHaCazado = False

# instalamos dependencias
!pip install treelib

from treelib import Node, Tree
from bs4 import BeautifulSoup
import requests
import random



# Función general => ojo se llama a si misma.
def busquedaGlobal(busquedasX, nivel):
  if googleNosHaCazado: return
  print("--------------------------")
  print(f"         Nivel {nivel}")
  print("--------------------------")

  subBusquedasRelacionadas = []
  
  for busqueda in busquedasX:
    subBusquedas = busquedaIndividual(busqueda, nivel+1)
    subBusquedasRelacionadas.extend(subBusquedas)
  
  if nivel < nivelesScrapeo:
    busquedaGlobal(subBusquedasRelacionadas, nivel +1) # con cuidado con bucle infinito

# función de scrapeo de cada búsqueda/pregunta
def busquedaIndividual(busqueda0X, nivel):
  busqueda0 = busqueda0X[0]
  url0 = busqueda0X[1] # opción url0 = f"https://www.google.com/search?hl={idioma}&gl={pais}&q={busqueda0}&oq={busqueda0}"
  
  global  googleNosHaCazado
  if busqueda0.lower() in scrapeado or googleNosHaCazado == True: return []
  
  preguntas = []
  busquedas = []
  
  tree = Tree() # arbol para reprensentarlo
  tree.create_node(busqueda0, busqueda0.lower())  # root node        
    
  wd.get(url0) # navegamos hasta la url
  
  if ("URL: https://www.google.com/search?" in wd.page_source): # ¿Nos ha cazado google?
    googleNosHaCazado = True

    if nivel == 1:
      print("-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
      print("Sorry Google ha cazado la IP, para seguir scrapeando elimina el entorno de ejecución (En el menú, Entorno de ejecución/desconectarse y eliminar...) o haz una copia de este colab (archivo/guardar una copia en drive) y dale al play")
      print("-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
    else:
      print("Google ens Roba, digo... Sorry Google nos ha cazado, fin del scrapeo de esta keyword")
    
    return []

  if nivel == 1:
    acepto = wd.find_elements(by=By.CSS_SELECTOR, value="input[value='Acepto']")
    if len(acepto) > 0:
      acepto[0].click() 

  comienza = random.randint(100, 500)/1000 # pongamos una pequeña espera random
  time.sleep(comienza)
  
  # recupero preguntas frecuentes
  PAA = wd.find_elements(by=By.CSS_SELECTOR, value="[class='xpc']") 
  if len(PAA) > 0 : tree.create_node("Preguntas Frecuentes", "Preguntas Frecuentes" , parent= busqueda0.lower())
  for boton in PAA:
    enlaces = boton.find_elements(by=By.TAG_NAME, value="a")
    for enlace in enlaces:
      href = enlace.get_attribute('href')
      if "/search" in href:
        contadorPreguntas.append(boton.text.strip().lower())
        if boton.text.strip() in tree: continue
        try:
          tree.create_node(boton.text.strip(), boton.text.strip().lower(), parent= "Preguntas Frecuentes")
        except:
          continue # en algunos casos excepcionales el texto a añadir al arbol es demasiado largo meda pereza investigarlos
        preguntas.append([boton.text.strip(), href])

  # recuperamos búsquedas relacionadas
  busquedasRelacionadas = wd.find_elements(by=By.CSS_SELECTOR, value=".Q71vJc") 
  if len(busquedasRelacionadas) > 0: tree.create_node("Búsquedas Relacionadas", "Búsquedas Relacionadas" , parent= busqueda0.lower())

  for busqueda in busquedasRelacionadas:
    contadorBusquedas.append(busqueda.text.strip().lower())
    if busqueda.text.strip() in tree: continue
    try:
      tree.create_node(busqueda.text.strip().lower(), busqueda.text.strip().lower(), parent="Búsquedas Relacionadas")
    except:
      continue
    busquedas.append([busqueda.text.strip(), busqueda.get_attribute('href')] )

  # recuperamos google suggest
  sugeridos = suggest(busqueda0)
  sugeridos = [x for x in sugeridos if x !=busqueda0 and x.lower() not in tree]
  if len(sugeridos) > 0 : tree.create_node("Google Suggest", "Google Suggest" , parent= busqueda0.lower())

  for sugerido in sugeridos:
    contadorSuggest.append(sugerido.lower())
    try:
      tree.create_node(sugerido, sugerido.lower(), parent="Google Suggest")
    except:
      continue
    
  if len(tree) > 1: tree.show(key=False)

  scrapeado.append(busqueda0.lower()) # control para no repetir búsquedas

  if loopPreguntasEnlugarDeBusquedas: return preguntas # opción devolver preguntas
  return busquedas # La opción por defecto es devolver búsquedas para el loop
 
# función para recuperar datos de google suggest
def suggest(key):
  r = requests.get(f'http://suggestqueries.google.com/complete/search?output=toolbar&hl={idioma}&gl={pais}&q={key}')
  soup = BeautifulSoup(r.content, 'html.parser')
  sugg = [sugg['data'] for sugg in soup.find_all('suggestion')]
  return sugg
    
      
# Aquí empieza todo
busquedaGlobal([[keyword, urlInicial]], 0)

# Mostramos tablas de resultados
from collections import Counter # contador para mostrar en tabla
contadorPreguntas1 = [list(x) for x in Counter(contadorPreguntas).most_common()]
contadorBusquedas1 = [list(x) for x in Counter(contadorBusquedas +contadorSuggest ).most_common()]


import pandas as pd
from google.colab import data_table
# Uso una función para visualizar tablas
def pasarATabla(lista,columnas):
  lista = pd.DataFrame (lista, columns = columnas )
  lista = data_table.DataTable(lista, include_index=True, num_rows_per_page=20)
  display(lista)

if len(contadorBusquedas1) > 0:
  print("---------------------------------")
  print("Siento una conmoción en la fuerza")
  print("----------------------------------------------------------------")
  print(f"Si quieres obtener más resultados prueba con nivelesScrapeo = {nivelesScrapeo+1}")
  print("----------------------------------------------------------------")
  pasarATabla(contadorBusquedas1, ["Búsqueda relacionada", "Relevancia"])

if len(contadorPreguntas1) > 0:
  pasarATabla(contadorPreguntas1, ["Preguntas relacionadas", "Relevancia"])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--------------------------
         Nivel 0
--------------------------


/usr/local/lib/python3.9/dist-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


tarjeta credito
├── Preguntas Frecuentes
│   ├── ¿Qué es y cómo funciona una tarjeta de crédito?
│   ├── ¿Cuál es el mejor banco para sacar una tarjeta de crédito?
│   ├── ¿Qué se necesita para tarjeta de crédito?
│   └── ¿Cuál es la tarjeta de crédito más barata en Argentina?
├── Búsquedas Relacionadas
│   ├── tarjeta de crédito solo con dni
│   ├── tarjeta de crédito visa
│   ├── www.supervielle.com.ar tarjeta de credito
│   ├── tarjeta de crédito online
│   ├── tarjeta de crédito galicia
│   ├── tarjeta de crédito banco nación
│   ├── solicitar tarjeta de crédito
│   └── tarjeta de crédito banco provincia
└── Google Suggest
    ├── tarjeta credito banco nacion
    ├── tarjeta credito banco provincia
    ├── tarjeta credito santander
    ├── tarjeta credito galicia
    ├── tarjeta credito bbva
    ├── tarjeta credito naranja
    ├── tarjeta credito macro
    ├── tarjeta credito santander rio
    └── tarjeta credito supervielle

--------------------------
         Nivel 1
------------

,Búsqueda relacionada,Relevancia
0,tarjeta de crédito,7
1,cual es la mejor tarjeta de crédito del mundo,6
2,tarjetas de créditos con mejores beneficios,5
3,tarjetas de crédito más usadas en argentina,5
4,que tarjeta de crédito me conviene,5
...,...,...
104,cual es la tarjeta de crédito más barata en ur...,1
105,cual es la tarjeta de crédito más barata en mé...,1
106,cual es la tarjeta de crédito más barata en perú,1
107,cual es la tarjeta de credito más barata,1


,Preguntas relacionadas,Relevancia
0,¿cuál es el mejor banco para sacar una tarjeta...,7
1,¿cuál es la tarjeta de crédito más barata en a...,7
2,¿cuáles son las tarjetas de crédito más usadas...,5
3,¿cómo puedo tener una tarjeta de crédito sin r...,2
4,¿cuál es la tarjeta de crédito más barata?,2
5,¿cómo saber si califico para tener una tarjeta...,2
6,¿qué es y cómo funciona una tarjeta de crédito?,1
7,¿qué se necesita para tarjeta de crédito?,1
8,¿cuál es la diferencia entre una tarjeta de cr...,1
9,¿cómo funciona el límite de la tarjeta de créd...,1
